In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.datasets.data_utils import get_file
import numpy as np
import random
import sys

Using Theano backend.
/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


Import text and get characters

In [2]:
text = open('SSE.txt').read().lower()
print('corpus length:', len(text))

chars = set(text)
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
log=""
for i, c in enumerate(chars):
    log+=c+" "
print (log)

corpus length: 6247475
total chars: 67

 !   # " % $ ' & ) ( + * - , / . 1 0 3 2 5 4 7 6 9 8 ; : = < ? > @ [ ] _ ^ a c b e d g f i h k j m l o n q p s r u t w v y x z | ~ 


cut text into length 20 sentences

In [3]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 20
num_sentences=1000000




#    for i in range(0,num_sentences):
#        r=random.randint(0,len(chars)-20)
#        sentences.append(text[r: r + maxlen])
#        next_chars.append(text[r + maxlen])
#    print('nb sequences:', len(sentences))
#    X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
#    y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
#    for i, sentence in enumerate(sentences):
#        for t, char in enumerate(sentence):
#            X[i, t, char_indices[char]] = 1
#        y[i, char_indices[next_chars[i]]] = 1
#    return X,y

Convert each char to 1-hot encoded vector
X: input 3-D Tensor, gives context for output
y: output 2-D Tensor, y[i,:] contains char that comes right after X[i,:,:]

In [4]:
#print('Vectorization...')
#X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
#y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
#for i, sentence in enumerate(sentences):
#    for t, char in enumerate(sentence):
#        X[i, t, char_indices[char]] = 1
#    y[i, char_indices[next_chars[i]]] = 1

build the model: 2 stacked LSTMs
incorporate dropout between each model for training regularization

In [5]:
print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(chars))))
#model.add(LSTM(512, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

Build model...


In [6]:
def sample(a, temperature=1.0):
    # helper function to sample an index from a probability array
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    return np.argmax(np.random.multinomial(1, a, 1))

train the model, output generated text after each iteration


In [ ]:
for iteration in range(1, 20):
    sentences = []
    next_chars = []
    for i in range(0,num_sentences):
        r=random.randint(0,len(text)-20)
        sentences.append(text[r: r + maxlen])
        next_chars.append(text[r + maxlen])
    print(sentences[0:10],next_chars[0:10])
    print('nb sequences:', len(sentences))
    X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
    y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            X[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1

    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=64, nb_epoch=1)
    model.save_weights('tb-weights', overwrite=True)
    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

['of exceptions (for e', " worrying that i'm a", 'hings like your son ', 'his is terribly rele', '-iq players had five', 'e is most classicall', 'l discrimination and', 'egime that rpeached ', 'ment against seeking', ' its predictive vali'] ['x', 'c', 'n', 'v', ' ', 'y', ' ', 't', ' ', 'd']
nb sequences: 1000000

--------------------------------------------------
Iteration 1
Epoch 1/1
   2560/1000000 [..............................] - ETA: 32395s - loss: 3.2842